# Boardgame Recommender

## 1. Business Understanding

**BoardGameGeek** is a comprehensive online database that consists of over 125,600 board games, providing access to reviews, discussion forums, and detailed information on individual titles. It is widely recognized as the most extensive repository of board game data available. In addition to serving as an information resource, the platform enables users to rate games on a 1–10 scale and to manage their personal game collections.

The goal of this project is to develop a recommender system that leverages data from BoardGameGeek to suggest board games to users. We chose an item-based collaborative filtering approach, using community rating data to find similarities between games. The system will generate personalized recommendations for selected user, based on their individual rating history.

## 2. Data Understanding

The dataset used in this project is the **Board Game Database** available on [Kaggle](https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek), which compiles data from the **BoardGameGeek (BGG)** platform. The data, collected in 2021, captures detailed information about board games, their attributes, and user interactions on the site. 

This dataset provides a rich foundation for exploring patterns in board game design, user preferences, and popularity, making it well-suited for building a **recommender system** based on user ratings and item similarities.

### 2.1 Overview of Available Files

The dataset consists of **nine files**, each offering different dimensions of board game data:

- **GAMES** – Contains core details for approximately **22,000 board games** with **47 features**.  
  Each game is identified by a unique **BGGId**, serving as the primary key. This file includes metadata such as publication year, minimum/maximum players, average playtime, and average user rating.

- **RATINGS_DISTRIBUTION** – Provides the **full rating distribution** for each game (`BGGId`), detailing how users rated the game on a 1–10 scale.

- **THEMES** – Lists **thematic categories** associated with each game (`BGGId`), allowing thematic-based filtering and analysis.

- **MECHANICS** – Includes **game mechanics** represented as binary indicators per game, enabling insights into gameplay styles (e.g., deck building, worker placement, area control).

- **SUBCATEGORIES** – Contains **secondary classifications** (binary flags) that describe additional attributes or gameplay aspects of each title.

- **ARTISTS_REDUCED** – Identifies **artists** involved in the visual design of each game.  
  Only artists with more than 3 works are included explicitly; others are represented by a binary flag.

- **DESIGNERS_REDUCED** – Provides information about **game designers**, using the same filtering logic as the artists file (designers with >3 works listed individually).

- **PUBLISHERS_REDUCED** – Contains **publisher information** with binary flags for those associated with fewer than 3 published games.

- **USER_RATINGS** – The largest and most critical table for recommendation modeling.  
  It contains user-generated ratings with over **411,000 unique users** and approximately **19 million ratings**, linking users to the games they have rated via `username` and `BGGId`.


### 2.2 Data Relevance to the Project

For this project, the **USER_RATINGS** dataset serves as the foundation for modeling user–game interactions and generating personalized recommendations.  
The accompanying **GAMES** dataset provides detailed metadata about each title and is used to filter and refine the set of games considered for recommendation.  
Other available files support the project but are not directly used in the core recommendation workflow.

### 2.3 Data Source

- **Primary Source:** [BoardGameGeek](https://boardgamegeek.com/)  
- **Dataset Repository:** [Board Games Database on Kaggle](https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek)

### 2.4 Data Access and Loading

To begin the analysis, the dataset is retrieved directly from **Kaggle** using the `kagglehub` library.  
This ensures easy and reliable access to the **Board Game Database** for exploration and modeling.

Install dependencies as needed:  
`pip install kagglehub[pandas-datasets]`

In [15]:
import kagglehub
import os
import pandas as pd

path = kagglehub.dataset_download("threnjen/board-games-database-from-boardgamegeek")

### 2.5 Exploring the `GAMES` File

The **`games.csv`** file serves as the **master information table**, containing detailed metadata for each board game listed on BoardGameGeek.  
Each game is uniquely identified by the `BGGId` and described using 47 attributes covering gameplay, community ratings, rankings, and publisher information.

Key features include:

- **BGGId** – Unique BoardGameGeek game identifier  
- **Name** – Title of the board game  
- **Description** – Lemmatized and punctuation-stripped description text  
- **YearPublished** – Year the game was first published  
- **GameWeight** – Difficulty or complexity rating  
- **AvgRating / BayesAvgRating** – Average and Bayesian-weighted average user ratings  
- **MinPlayers / MaxPlayers / BestPlayers** – Recommended and community-voted player counts  
- **ComAgeRec / MfgAgeRec** – Community and manufacturer age recommendations  
- **MfgPlayTime / ComMinPlaytime / ComMaxPlaytime** – Estimated and community-reported play times  
- **NumOwned / NumWant / NumWish** – Ownership and wishlist statistics  
- **Rank:** fields – Game rankings across multiple categories (e.g., strategy, thematic, party)  
- **Cat:** fields – Binary indicators for major game categories  

The following code loads the `games.csv` file into a pandas DataFrame and displays the first few records for inspection.


In [16]:
df_games = pd.read_csv(os.path.join(path, "games.csv"))
display(df_games.head())

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0


### 2.7 Exploring the `USER_RATINGS` File

The **`user_ratings.csv`** file captures individual user ratings for board games listed on BoardGameGeek.  
Each record links a **user** to a **game** through its unique `BGGId` and the corresponding numeric rating, forming the foundation for collaborative filtering–based recommendations.

Key features include:

- **BGGId** – Unique identifier for each board game (foreign key referencing the `GAMES` table)  
- **Rating** – The raw user-assigned rating on a 1–10 scale  
- **Username** – Identifier of the user who provided the rating  

This dataset contains over **19 million ratings** from approximately **411,000 unique users**, making it the **core interaction data** for the recommender system.

The following code loads the `user_ratings.csv` file into a pandas DataFrame and displays the first few records to examine its structure.


In [17]:
df_user_ratings = pd.read_csv(os.path.join(path, "user_ratings.csv"))
display(df_user_ratings.head())

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP


### 2.6 Data Quality Assessment

A thorough assessment of data quality was conducted to ensure the dataset is suitable for analysis and modeling.  
This evaluation focused on two key aspects: **missing values** and **outliers**.  
The findings are based on the official dataset documentation provided on Kaggle:  
[Board Games Database (BoardGameGeek, Kaggle)](https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek?select=games.csv)

#### Missing Values
- The **`games.csv`** file was found to be **largely complete**, with **only one missing value** detected in the `Description` column.  
  Since this column contains textual metadata and is not critical for the recommendation system, this missing entry does not pose a problem for the analysis.
- The **`user_ratings.csv`** file contains **no missing values**, ensuring the user–item interaction data is fully populated and ready for modeling.

#### Outliers
- Based on both the exploratory inspection and the accompanying dataset documentation, the **`games.csv`** file contains **no outlier values** in its numerical attributes (e.g., `AvgRating`, `GameWeight`, or `NumUserRatings`).
- The **`user_ratings.csv`** file also shows **no outliers**, with all ratings conforming to the expected **1–10** user rating scale.

Overall, both datasets demonstrate **excellent data quality**, requiring minimal cleaning prior to the development of the recommender system.

## 3. Data Preparation

 cleanataan data 

In [18]:
# Target user
target_user = "TeemuVataja"

In [19]:
from surprise import Reader, Dataset

# Example df_user_ratings has columns: Username, Item, Rating

# Build user table with unique IDs and rating counts
# If we ever need a dataframe without ratings just copy df_users before this step
df_users = (
    df_user_ratings.groupby("Username")
    .size()  # counts ratings per user
    .reset_index(name="RatingCount")
    .reset_index(names="UserId")  # create UserId from row index
)

#TODO: printtejä

print("Total number of users:")
display(len(df_users))

print("User table")
display(df_users.head())

print("User rating table")
display(df_user_ratings.head())

print("Selected user table")
display(df_users[df_users["Username"] == target_user])

Total number of users:


411374

User table


,UserId,Username,RatingCount
0,0,Fu_Koios,2
1,1,beastvol,9
2,2,mycroft,14
3,3,woh,5
4,4,(mostly) harmless,1


User rating table


,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP


Selected user table


,UserId,Username,RatingCount
187874,187874,TeemuVataja,206


Users with more than equal 10 ratings

In [20]:
print(f"Length of users set before reduction: {len(df_users)}")

df_users_reduced = df_users[df_users['RatingCount'] >= 10]

print(f"Length of users set after reduction: {len(df_users_reduced)}")

Length of users set before reduction: 411374
Length of users set after reduction: 224604


In [21]:
print(f"Length of games set before reduction: {len(df_games)}")

df_games_reduced = df_games[df_games['NumUserRatings'] >= 100]

print(f"Length of games set after reduction: {len(df_games_reduced)}")

Length of games set before reduction: 21925
Length of games set after reduction: 12239


In [22]:
print(f"Length of ratings set before reduction: {len(df_user_ratings)}")

df_ratings_reduced = df_user_ratings[df_user_ratings["BGGId"].isin(df_games_reduced["BGGId"])]
df_ratings_reduced = df_ratings_reduced[df_ratings_reduced["Username"].isin(df_users_reduced["Username"])]

print(f"Length of ratings set after reduction: {len(df_ratings_reduced)}")
print(f"Valid ratings after pruning for target user: {len(df_ratings_reduced[df_ratings_reduced['Username'] == target_user])}")

Length of ratings set before reduction: 18942215
Length of ratings set after reduction: 17849716
Valid ratings after pruning for target user: 198


## 4. Modeling


Selitä mitä nyt tehdään
# SVD = item x item
# KNN = user x user
Mallin luonti ja tallennus tietokoneelle. Jos ei luotu niin kouluttaa sen ja tallentaa

In [23]:
from surprise import SVD, dump
from surprise.model_selection import train_test_split
import os

def create_surprise_dataset(df, user_id, item_id, ratings, reader):
    return Dataset.load_from_df(df[[user_id, item_id, ratings]], reader)

data = create_surprise_dataset(df_ratings_reduced, "Username", "BGGId", "Rating", Reader(rating_scale=(0.0, 10.0)))

# def load_test_model(model_name, algo, force_recreate=False):
#     if not os.path.exists(model_name) or force_recreate:
#         surprise_dataset = create_surprise_dataset(df_ratings_reduced, "Username", "BGGId", "Rating", Reader(rating_scale=(0.0, 10.0))) # First create surprise dataset from an existing Pandas Dataframe. Hardcoded for now
#         trainset, testset = train_test_split(surprise_dataset, test_size=0.2)
#         algo.fit(trainset)
#         #surprise_trainset = surprise_dataset.build_full_trainset() # Then build the trainset
#         #algo.fit(surprise_trainset) # Lastly create the model using said training set
#         
#         dump.dump(model_name, algo=algo)
#         return algo, testset
# 
#     _, algo = dump.load(model_name)
#     return algo
# 
# model, testset = load_test_model("testmodel.pkl", SVD(n_factors=50, random_state=666))

predictions funktiot.

In [24]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def top_similar_users(username, model, top_n=10):
    """
    Returns top-N similar users to a given username (raw ID).
    """
    try:
        inner_uid = model.trainset.to_inner_uid(username)
    except ValueError:
        return []

    user_vec = model.pu[inner_uid].reshape(1, -1)
    sims = cosine_similarity(user_vec, model.pu)[0]

    # Exclude self
    top_inner = np.argsort(-sims)[1:top_n+1]
    return [model.trainset.to_raw_uid(i) for i in top_inner]


Selitystä

In [25]:

def user_predictions(username, model, top_n_items=None):
    """
    Predict ratings for all unseen items for a given username.
    Optionally, return only top-N items.
    """
    try:
        inner_uid = model.trainset.to_inner_uid(username)
    except ValueError:
        return []

    # Items already rated by user
    user_items = {iid for (iid, _) in model.trainset.ur[inner_uid]}
    all_items = set(range(model.trainset.n_items))
    unseen_items = all_items - user_items

    # Build test set (user x unseen items)
    testset = [(username, model.trainset.to_raw_iid(i), 0.) for i in unseen_items]

    # Get predictions
    predictions = model.test(testset)

    # Optionally keep top-N items
    if top_n_items:
        predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n_items]

    return predictions


Selitystä

In [26]:
def user_predictions_from_similar(username, model, top_n_users=1000, top_n_items=10):
    """
    Predict items for a user based on items liked by similar users.
    """
    similar_users = top_similar_users(username, model, top_n=top_n_users)

    # Collect items rated by similar users
    similar_items = set()
    for su in similar_users:
        inner_uid = model.trainset.to_inner_uid(su)
        similar_items.update([model.trainset.to_raw_iid(iid) for (iid, _) in model.trainset.ur[inner_uid]])

    # Filter out items already rated by target user
    inner_uid_target = model.trainset.to_inner_uid(username)
    user_items = {model.trainset.to_raw_iid(iid) for (iid, _) in model.trainset.ur[inner_uid_target]}
    candidate_items = similar_items - user_items

    # Build test set and predict
    testset = [(username, iid, 0.) for iid in candidate_items]
    predictions = model.test(testset)
    predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n_items]

    return predictions


Selitystä

## 5. Evaluation

The performance of the recommendation model in this project is evaluated using **cross-validation**.  
Specifically, the `cross_validate` function from the `surprise.model_selection` module is employed to assess the **SVD (Singular Value Decomposition)** model.

In this setup, the SVD model is configured with 50 latent factors (`n_factors=50`) and a fixed random seed (`random_state=666`) to ensure reproducibility.  
A three-fold cross-validation (`cv=3`) is performed, meaning the dataset is divided into three equal parts. In each iteration, two-thirds of the data are used to train the model, while the remaining one-third is used for testing.  

Model performance is measured using two key metrics: **Root Mean Square Error (RMSE)** and **Mean Absolute Error (MAE)**. These metrics are computed for both the training and test sets (`return_train_measures=True`), allowing us to evaluate not only how well the model fits the data it was trained on, but also how effectively it generalizes to unseen data.  

Overall, this evaluation approach provides a reliable estimate of the model’s predictive accuracy and its ability to generalize beyond the training data.


In [27]:
from surprise.model_selection import cross_validate

results = cross_validate(
    SVD(n_factors=50, random_state=666),
    data, measures=["RMSE", "MAE"],
    cv=3,
    return_train_measures=True,
    verbose=True
)

results_df = pd.DataFrame(results)
results_df

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1587  1.1590  1.1579  1.1585  0.0005  
MAE (testset)     0.8650  0.8654  0.8648  0.8651  0.0003  
RMSE (trainset)   0.8903  0.8896  0.8896  0.8898  0.0003  
MAE (trainset)    0.6744  0.6739  0.6738  0.6740  0.0003  
Fit time          140.05  151.23  98.62   129.97  22.63   
Test time         65.41   52.73   54.99   57.71   5.52    


,test_rmse,train_rmse,test_mae,train_mae,fit_time,test_time
0,1.158669,0.890270,0.865032,0.674439,140.054871,65.406096
1,1.158991,0.889581,0.865449,0.673876,151.225025,52.729806
2,1.157884,0.889636,0.864770,0.673824,98.617773,54.990134


### Understanding the Performance Metrics

#### **Root Mean Square Error (RMSE)**  

   $$RMSE = \sqrt{\frac{1}{N} \sum_{(u,i)} (\hat{r}_{ui} - r_{ui})^2}$$  

RMSE measures the square root of the average squared difference between predicted and actual ratings.

Because the errors are **squared before averaging**, RMSE gives **more weight to larger errors** — meaning a few big mistakes will increase this value significantly.  

For board game recommendation systems using a **1–10 rating scale**, typical RMSE values range from **1.0 to 2.0**:  
- **< 1.0** → Excellent predictive performance  
- **1.0–1.5** → Good, typical for collaborative filtering models  
- **> 2.0** → Poor, indicating weak predictive accuracy or sparse data  

A **lower RMSE** means the model’s predictions are generally close to actual ratings, while a higher value suggests inconsistent or inaccurate predictions. 

RMSE is especially useful when you want to **penalize large errors**, such as recommending something that a user would really dislike.

    
#### **Mean Absolute Error (MAE)**
 
   $$MAE = \frac{1}{N} \sum_{(u,i)} |\hat{r}_{ui} - r_{ui}|$$  

MAE represents the **average absolute difference** between predicted and actual ratings, treating all errors equally.  
It is more **robust to outliers** and easier to interpret: it tells you, on average, how far your predictions are from the true ratings.  

For the same 1–10 scale, **MAE** values are typically **slightly lower than RMSE**:  
- **< 1.0** → Very good accuracy  
- **1.0–1.5** → Acceptable, common for many recommenders  
- **> 1.5** → Mediocre, suggesting inconsistent predictions  

A **lower MAE** indicates more stable and reliable performance overall.

In [28]:
# Display mean performance metrics for quick summary
mean_results = results_df.mean().rename("Mean Value")
mean_results[['test_rmse', 'train_rmse', 'test_mae', 'train_mae']]


test_rmse     1.158515
train_rmse    0.889829
test_mae      0.865083
train_mae     0.674046
Name: Mean Value, dtype: float64

## 6. Deployment

In [29]:
def load_surprise_model(model_name, algo, force_recreate=False):
    if not os.path.exists(model_name) or force_recreate:
        surprise_dataset = create_surprise_dataset(df_ratings_reduced, "Username", "BGGId", "Rating", Reader(rating_scale=(0.0, 10.0))) # First create surprise dataset from an existing Pandas Dataframe. Hardcoded for now
        surprise_trainset = surprise_dataset.build_full_trainset() # Then build the trainset
        algo.fit(surprise_trainset) # Lastly create the model using said training set
        
        dump.dump(model_name, algo=algo)
        return algo

    _, algo = dump.load(model_name)
    return algo

model = load_surprise_model("model.pkl", SVD(n_factors=50,random_state=666))

##tehdään tällä juttutja

tähän juttuja mallin luonnista

In [30]:
#TODO: poista sarakkeita tulostuksesta
# Example usage
predictions = user_predictions_from_similar(target_user, model)

top10 = sorted(predictions, key=lambda x: x.est, reverse=True)
for pred in top10:
    print(pred.iid, pred.est)
    display(df_games_reduced[df_games_reduced["BGGId"] == pred.iid])
    

262211 9.365823353856806


,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
19434,262211,Cloudspire,cloudspire player strategy game heavily infl...,2019,4.2794,8.26495,6.90388,1.63067,1,4,...,21926,21926,0,1,0,0,0,0,0,0


210232 9.244231524173466


,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
16693,210232,Dungeon Degenerates: Hand of Doom,dungeon degenerates hand doom take place dark ...,2017,3.5574,8.23936,6.27133,1.69541,1,4,...,21926,21926,1,0,0,0,0,0,0,0


218421 9.235776785392913


...

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
18811,252328,Star Wars: X-Wing (Second Edition),xwe second edition put command squadron advanc...,2018,3.0435,8.23292,6.87496,1.56988,2,2,...,21926,21926,0,0,1,0,1,0,0,0


342942 9.068026220319174


,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
21896,342942,Ark Nova,ark nova plan design modern scientifically man...,2021,3.7714,8.47839,6.25826,1.38243,1,4,...,21926,21926,0,1,0,0,0,0,0,0


184267 8.987256409580551


,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
15209,184267,On Mars,follow success unmanned rover mission united n...,2020,4.6477,8.25962,7.7036,1.49724,1,4,...,21926,21926,0,1,0,0,0,0,0,0


ajatuksia